imports

In [1]:
from tqdm.notebook import tqdm as tqdm
import os
import json
import pandas as pd

from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from myhelpers import TrialStatistics

parameters

In [2]:
experimentsPath="/home/elhamod/HGNN/experiments/"
dataPath="/data/BGNN_data"
experimentName="Testing_layer_cases_TwoNets"

cuda=6

file and changes

In [3]:
old_params_file = os.path.join(experimentsPath, experimentName, "params.json")

# Keep same order as ConfigParserWriter!
changes = {
    "aug_profile": "withRotation_PCA1.5"
}

prepare

In [4]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)

# populate a hash table of all experiments, indexed by trial hash
experiments = {}
for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
    num_of_models = experiment_params["numOfTrials"]
    for k in range(num_of_models):
        old_trialHash = TrialStatistics.getTrialName(experiment_params, k)
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        new_trialHash = TrialStatistics.getTrialName(experiment_params_new, k)
        experiments[old_trialHash] = {"trialHash": new_trialHash}

Fix data splits

In [5]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        # update experiment params
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        # fix data split names
        old_dataset_name = getDatasetName(config_parser.fixPaths(experiment_params))
        old_datasplitsPath = os.path.join(experimentsPath, experimentName, old_dataset_name)
        new_dataset_name = getDatasetName(config_parser.fixPaths(experiment_params_new))
        new_datasplitsPath = os.path.join(experimentsPath, experimentName, new_dataset_name)
        
        try:
            if os.path.exists(old_datasplitsPath):

                # update the folder names
                os.rename(old_datasplitsPath, new_datasplitsPath)
                print("datasplit", old_datasplitsPath, '->', new_datasplitsPath, "updated")

                # update the params.json
                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_datasplitsPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
                
            else:
                raise
                
        except:
            print("datasplit", old_dataset_name, "could not be fixed to", new_dataset_name)
            pass   
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['datasetName'] = new_dataset_name

        
        bar.update()

datasplit /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a -> /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a updated
json /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a/params.json updated
datasplit /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a -> /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a updated
json /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a/params.json updated
datasplit /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a -> /home/

Fix models

In [6]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        num_of_models = experiment_params["numOfTrials"]
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for k in range(num_of_models):
            
                # Fix model names
                old_model_name = getModelName(experiment_params, k)
                old_modelPath = os.path.join(experimentsPath, experimentName, old_model_name)
                new_model_name = getModelName(experiment_params_new, k)
                new_modelPath = os.path.join(experimentsPath, experimentName, new_model_name)

                try:
                    if os.path.exists(old_modelPath):
                        os.rename(old_modelPath, new_modelPath)
                        print("model", old_modelPath, '->', new_modelPath, "updated")

                        j = json.dumps(experiment_params_new)
                        json_fileName = os.path.join(new_modelPath, 'params.json')
                        f = open(json_fileName,"w")        
                        f.write(j)
                        f.close()  
                        print("json", json_fileName, "updated")
                    else:
                        raise
                    
                except:
                    print("model", old_model_name, "could not be fixed to", new_model_name)
                    pass  
                
                

                bar2.update()
                
                experiments[TrialStatistics.getTrialName(experiment_params, k)]['modelName'] = new_model_name

        bar.update()

model /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/models/8d2e4649ff9d50a6ab53dcbab9cf799e82237cb3186969410ec2f762 -> /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/models/27398c7fd0bc2232f63885417855711b5a5cff5718b4f7f699950cfd updated
json /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/models/27398c7fd0bc2232f63885417855711b5a5cff5718b4f7f699950cfd/params.json updated



model /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/models/7df7d1761492fab3535cf32c4335fd438877d21b34a5b4fb7d10ea1d -> /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/models/0f1d9868bbb1924250f2e1895688c41e6fd6106101ed858026048f0f updated
json /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/models/0f1d9868bbb1924250f2e1895688c41e6fd6106101ed858026048f0f/params.json updated



model /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/models/47ba45e62605b18418cae49e2cd88bce710d67a09f7f0de24a44e02f -> /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/models/6bc7a3a3fa6a4c4ecdcc3a1ef7e26a2dd724db453e2fc55b578d82df updated
json /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/models/6bc7a3a3fa6a4c4ecdcc3a1ef7e26a2dd724db453e2fc55b578d82df/params.json updated



model /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/models/b801207da7faf4715e979d24a7c2be622657ec5477c50b639f508fb6 -> /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/models/31510c5322e87075e6e794bae5ad3b46ba71dda99e7e7ef87b309b71 updated
json /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/models/31510c5322e87075e6e794bae5ad3b46ba71dda99e7e7ef87b309b71/params.json updated




Fix results directory

In [7]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        old_experimentHash = TrialStatistics.getTrialName(experiment_params)
        old_experimentPath = os.path.join(experimentsPath, experimentName, "results", old_experimentHash)
        new_experimentHash = TrialStatistics.getTrialName(experiment_params_new)
        new_experimentPath = os.path.join(experimentsPath, experimentName, "results", new_experimentHash)
        
        try:
            if os.path.exists(old_experimentPath):

                os.rename(old_experimentPath, new_experimentPath)
                print("result", old_experimentPath, '->', new_experimentPath, "updated")

                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_experimentPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
            else:
                raise
                
        except:
            print("result", old_experimentHash, "could not be fixed to",new_experimentHash)
            pass 
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['experimentHash'] = new_experimentHash
        
        bar.update()

result 6f4c6ea6c93f6d0d1a1554cf9d4a1cdbc8ee29e6b120944ce5dc881f could not be fixed to 125ee08dfe6ba23fed5fb8941847bda019ea9828c2e6abeabb6d83ba
result 8a3b8b399c79da7ffb25aba02fc0b48f92ff52eb763e78dc632c654c could not be fixed to 7be342dc25f14a31a356c193e1f24c20e8e0311ab8175ee2eca5de27
result 3c36562a081550a97b4fdc9deeb8760157909672d842f600fa003ac5 could not be fixed to 020252a5c8a2ac261348f2cdef09e1b0f115638ce65cd98222056b90
result 10c137a24ea9c7cea5f3b5b82ce17fc354a7e77fbb90943a8ac12550 could not be fixed to a4d863db4d407868fd104061f534dae975774280842a71e1c02daee3



Fix experiments file

In [8]:
# open experiments.csv file
experimentsFileNameAndPath=os.path.join(experimentsPath, "experiments.csv")
experiments_file= pd.read_csv(experimentsFileNameAndPath)

# For all experiments
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        numOfTrials = experiment_params["numOfTrials"]
        trialHash_0 = TrialStatistics.getTrialName(experiment_params, 0) # used for keys who don't change for different trials
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        
        # For all trials
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for j in range(numOfTrials):
                
                trialHash = TrialStatistics.getTrialName(experiment_params, j)
                try:
                    # If old entry doesn't exist, nothing to update
                    record_exists = not (experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName]).empty
                    if record_exists:
                        
                        # drop old row and caluclate new row
                        new_trialHash = experiments[trialHash]["trialHash"]
                        experiments_file.drop(experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 
                        row_information = {
                            'experimentName': experimentName,
                            'modelName': experiments[trialHash]["modelName"],
                            'datasetName': experiments[trialHash_0]["datasetName"],
                            'experimentHash': experiments[trialHash_0]["experimentHash"],
                            'trialHash': new_trialHash
                        }
                        row_information = {**row_information, **experiment_params_new} 

                        # error handling to avoid duplicates
                        record_exists = not (experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName]).empty
                        if record_exists:
                            experiments_file.drop(experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 

                        # update with new entry    
                        experiments_file = experiments_file.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
                        print("trialHash", trialHash, "->", new_trialHash, "updated")
                except:
                    print("trialHash", trialHash, "could not be updated!")
                    pass
                bar2.update()
        bar.update()

# resave file
experiments_file.to_csv(experimentsFileNameAndPath, header=True, index=False)

trialHash 8d2e4649ff9d50a6ab53dcbab9cf799e82237cb3186969410ec2f762 -> 27398c7fd0bc2232f63885417855711b5a5cff5718b4f7f699950cfd updated



/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Fix params file

In [9]:
experiment_list = []
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_list.append(experiment_params_new)
        bar.update()


j = json.dumps({"experimentList": experiment_list})
f = open(old_params_file,"w")        
f.write(j)
f.close()  
print("json", old_params_file, "updated")


json /home/elhamod/HGNN/experiments/Testing_layer_cases_TwoNets/params.json updated
